In [ ]:
import pickle

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from tqdm import tqdm

In [ ]:
# unpickle data
with open('training_data_N100000.pickle', 'rb') as f:
    emp = pickle.load(f)


# pickle stores data as a dictionary
# this dictionary has 3 keys; the values are lists of lists
# 10,000 lists with 40 values in each list
# we are only interested in the "train_input_sequence"
# save the values from the "train_input_sequence"
value = emp.get("train_input_sequence")

In [ ]:
# convert list of lists in pandas dataframe
chaos_df = pd.DataFrame.from_records(value)
chaos_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39
0,0.279228,8.091169,1.098011,-0.463161,3.888320,8.665831,4.122427,3.013807,-2.558740,2.015656,-1.454905,-3.492466,3.817715,15.570893,2.318342,3.349483,-2.163914,-1.420784,-3.493829,1.104612,2.208701,1.523153,3.779100,10.088348,-0.110143,-1.277557,4.562407,4.596546,-3.042859,3.183928,3.194883,8.567653,-4.729759,0.054221,0.406357,3.280737,4.744961,7.251823,-2.394936,2.396356
1,0.639190,8.103688,1.114733,-0.405329,3.916085,8.855433,4.086295,2.622038,-2.489846,2.201283,-1.360045,-3.378496,3.296093,15.719268,2.335934,3.003640,-2.157252,-1.164615,-3.401325,1.070747,2.340784,1.670555,3.964520,10.013765,-0.518284,-1.148164,4.557367,4.568919,-2.973412,3.286775,3.631329,8.309227,-4.845136,0.533307,0.525735,3.368705,5.017679,7.000191,-2.439430,2.632433
2,1.020907,8.109028,1.105917,-0.348018,3.947265,9.040057,4.011209,2.235900,-2.408282,2.370115,-1.266747,-3.255328,2.798576,15.834258,2.378263,2.653162,-2.131953,-0.921544,-3.301191,1.044601,2.470625,1.825347,4.155992,9.913809,-0.922796,-0.998443,4.555152,4.538549,-2.899794,3.374232,4.062660,8.010899,-4.947711,0.995952,0.683484,3.460898,5.280053,6.728976,-2.479989,2.853503
3,1.422363,8.105092,1.070217,-0.290007,3.981976,9.218281,3.896544,1.860655,-2.315637,2.522886,-1.174799,-3.124848,2.327383,15.918859,2.440564,2.295181,-2.090417,-0.692572,-3.194081,1.025737,2.598601,1.987915,4.352890,9.786361,-1.320908,-0.829877,4.557959,4.504671,-2.822918,3.447214,4.485192,7.673067,-5.037566,1.436631,0.879364,3.558367,5.529729,6.438319,-2.516258,3.057959
4,1.841218,8.089587,1.006697,-0.230055,4.020474,9.388586,3.742280,1.501600,-2.213724,2.660626,-1.084034,-2.988791,1.884208,15.975813,2.517204,1.927703,-2.034504,-0.478365,-3.080724,1.013676,2.725084,2.158627,4.554402,9.629341,-1.709887,-0.644488,4.568065,4.466512,-2.743724,3.506743,4.895504,7.296767,-5.114860,1.849911,1.112464,3.661977,5.764224,6.128518,-2.547773,3.244287
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,1.060426,5.989071,6.247471,-5.152830,-3.157959,-1.220203,4.338162,7.673255,8.757387,4.786347,-5.615753,2.634875,2.750387,4.212863,8.156296,1.013503,1.332969,5.481186,4.736040,-4.034887,3.426418,2.450849,3.570506,0.789801,1.046649,7.532481,9.264381,-3.524874,5.483455,15.669940,2.116784,-0.850408,2.250179,2.714730,1.591655,1.194200,2.349632,5.606107,10.276412,2.194318
99996,1.065238,6.072060,5.899901,-5.541860,-2.633883,-1.386565,4.254071,8.127178,8.763468,3.654184,-5.714779,2.781539,3.092499,4.427563,8.089422,0.863958,1.394922,5.576008,4.492691,-3.988338,3.571460,2.790045,3.564272,0.827730,1.168133,7.648268,8.919529,-3.589337,5.291559,15.893294,1.204494,-0.965247,2.316958,2.844904,1.646298,1.275909,2.476367,5.866608,10.249753,1.794484
99997,1.089245,6.155343,5.527820,-5.869672,-2.101007,-1.506767,4.156526,8.571895,8.681358,2.493926,-5.739380,2.842160,3.449406,4.653021,7.996090,0.705794,1.461594,5.670944,4.245870,-3.935679,3.690944,3.133598,3.546059,0.857700,1.290857,7.768230,8.557568,-3.673348,5.076812,16.061063,0.312815,-0.958510,2.372558,2.977835,1.699605,1.356563,2.607912,6.132979,10.190087,1.375291
99998,1.132336,6.239663,5.133349,-6.138031,-1.569429,-1.576776,4.051260,9.000456,8.502155,1.327043,-5.692411,2.815445,3.810664,4.890466,7.873172,0.539722,1.532765,5.766107,3.995626,-3.878132,3.785410,3.478772,3.515807,0.880016,1.414700,7.891334,8.176435,-3.776176,4.838429,16.178954,-0.531715,-0.833693,2.420084,3.110459,1.751899,1.436239,2.744603,6.404370,10.093426,0.940331


In [ ]:
#normalize data
scaler = MinMaxScaler(feature_range=(-1,1))
scaled_data = scaler.fit_transform(chaos_df.values)

In [ ]:
data_small = scaled_data[:10000]
data_small

array([[-1.63308990e-01,  3.90806422e-01, -1.30601614e-01, ...,
         2.63799899e-01, -2.31968033e-01,  1.27087305e-02],
       [-1.37082800e-01,  3.91672635e-01, -1.29415551e-01, ...,
         2.45688538e-01, -2.35111804e-01,  3.12882219e-02],
       [-1.09271608e-01,  3.92042118e-01, -1.30040884e-01, ...,
         2.26167659e-01, -2.37977591e-01,  4.86867175e-02],
       ...,
       [-6.21411443e-01,  7.29255281e-02,  6.78304882e-01, ...,
        -3.29649210e-01,  3.35461019e-01,  5.40593206e-02],
       [-6.14718909e-01,  3.68425173e-02,  7.05335756e-01, ...,
        -3.17936378e-01,  3.37236912e-01,  3.72690666e-02],
       [-6.08770755e-01, -5.62800420e-04,  7.27397850e-01, ...,
        -3.06073959e-01,  3.39377504e-01,  2.01928066e-02]])

In [ ]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x = sequences[i:end_ix-1] 
        seq_y = sequences[end_ix-1]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)


In [ ]:
class MV_LSTM(torch.nn.Module):
    def __init__(self,n_features,seq_length):
        super(MV_LSTM, self).__init__()
        self.n_features = n_features
        self.seq_len = seq_length
        self.n_hidden = 150 # number of hidden states
        self.n_layers = 1 # number of LSTM layers (stacked)
    
        self.l_lstm = torch.nn.LSTM(input_size = n_features, 
                                 hidden_size = self.n_hidden,
                                 num_layers = self.n_layers, 
                                 batch_first = True)
        # according to pytorch docs LSTM output is 
        # (batch_size,seq_len, num_directions * hidden_size)
        # when considering batch_first = True
        self.l_linear = torch.nn.Linear(in_features=self.n_hidden*(seq_length-1), out_features=40)
        
    
    def init_hidden(self, batch_size):
        # even with batch_first = True this remains same as docs
        hidden_state = torch.zeros(self.n_layers,batch_size,self.n_hidden)
        cell_state = torch.zeros(self.n_layers,batch_size,self.n_hidden)
        self.hidden = (hidden_state, cell_state)
    
    
    def forward(self, x):        
        batch_size, seq_len,_ = x.size()
        
        lstm_out, self.hidden = self.l_lstm(x,self.hidden)
        # lstm_out(with batch_first = True) is 
        # (batch_size,seq_len,num_directions * hidden_size)
        # for following linear layer we want to keep batch_size dimension and merge rest       
        # .contiguous() -> solves tensor compatibility error
        x = lstm_out.contiguous().view(batch_size,-1)
        return self.l_linear(x)

In [ ]:
n_features = 40 # this is number of parallel inputs
n_timesteps = 5 # this is number of timesteps

# convert dataset into input/output
X, y = split_sequences(data_small, n_timesteps)
print(X.shape, y.shape)

# create NN
mv_net = MV_LSTM(n_features,n_timesteps)
criterion = torch.nn.MSELoss() # reduction='sum' created huge loss value
optimizer = torch.optim.Adam(mv_net.parameters(), lr=1e-1)


(9996, 4, 40) (9996, 40)


In [ ]:
batch_size = 4
epochs = 50
epochs = int(epochs)

In [ ]:
losses = []
min_val_loss = np.Inf
n_epochs_stop = 8
early_stop = False

mv_net.train()
for t in tqdm(range(epochs)):
  val_loss = 0
  for b in range(len(X)):
      inpt = X[b:b+batch_size,:,:]
      target = y[b:b+batch_size]    
      
      optimizer.zero_grad()

      x_batch = torch.tensor(inpt,dtype=torch.float32)    
      y_batch = torch.tensor(target,dtype=torch.float32)
  
      mv_net.init_hidden(x_batch.size(0))
      output = mv_net(x_batch) 
      loss = criterion(output, y_batch)  
      
      loss.backward()
      optimizer.step() 


      if loss.item() < min_val_loss:
        epochs_no_improve = 0
        min_val_loss = loss.item()
      else:
        epochs_no_improve += 1
      
      if t > 5 and epochs_no_improve == n_epochs_stop:
        print("Stopping early!")
        early_stop = True
        break
      else:
        continue
  if early_stop:
      print("Stopped")
      break
  print('\nEpoch: ' , t , '; loss: ' , loss.item())
  losses.append(loss.item())




  0%|          | 0/50 [00:00<?, ?it/s]


  2%|▏         | 1/50 [02:42<2:13:05, 162.98s/it]


Epoch:  0 ; loss:  0.00010597370419418439





  4%|▍         | 2/50 [05:34<2:12:19, 165.40s/it]


Epoch:  1 ; loss:  0.000133456225739792





  6%|▌         | 3/50 [08:30<2:12:13, 168.79s/it]

...


Epoch:  10 ; loss:  0.00024286043480969965





 24%|██▍       | 12/50 [41:33<2:19:51, 220.83s/it]


Epoch:  11 ; loss:  0.00027401273837313056





 26%|██▌       | 13/50 [45:17<2:16:53, 221.98s/it]


Epoch:  12 ; loss:  2.8109741833759472e-05
Stopping early!
Stopped


In [ ]:
from google.colab import files
torch.save(mv_net.state_dict(), 'model2.pth')

# download checkpoint file
files.download('model2.pth')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
model = MV_LSTM(n_features, n_timesteps)
model.load_state_dict(torch.load('model2.pth'))
model.eval()

MV_LSTM(
  (l_lstm): LSTM(40, 150, batch_first=True)
  (l_linear): Linear(in_features=600, out_features=40, bias=True)
)

In [ ]:
test_data = scaled_data[20000:25000]
X_test, y_test = split_sequences(test_data, n_timesteps)

In [ ]:
accuracy = []
for b in tqdm(range(len(X_test))):
  inpt = X[b:b+1,:,:]
  target = y[b:b+1]    
  
  x_batch = torch.tensor(inpt,dtype=torch.float32)    
  y_batch = torch.tensor(target,dtype=torch.float32)

  model.init_hidden(x_batch.size(0))
  output = model(x_batch)

  output = output.cpu().detach().numpy()
  acc = []
  for i,_ in enumerate(output):
    acc.append(np.abs((output[i] - target[i])/target[i]))

  rolling_acc = np.sum(acc)/40
  accuracy.append(rolling_acc)
  





  0%|          | 0/4996 [00:00<?, ?it/s]



  3%|▎         | 142/4996 [00:00<00:03, 1411.39it/s]



  5%|▌         | 252/4996 [00:00<00:03, 1298.72it/s]



  8%|▊         | 376/4996 [00:00<00:03, 1278.17it/s]



 10%|▉         | 492/4996 [00:00<00:03, 1237.45it/s]



 12%|█▏        | 610/4996 [00:00<00:03, 1216.26it/s]



 14%|█▍        | 717/4996 [00:00<00:03, 1166.88it/s]



 16%|█▋        | 823/4996 [00:00<00:03, 1132.23it/s]



 19%|█▉        | 942/4996 [00:00<00:03, 1148.31it/s]



 21%|██        | 1050/4996 [00:00<00:03, 1121.26it/s]



 23%|██▎       | 1158/4996 [00:01<00:03, 1047.04it/s]



 26%|██▌       | 1287/4996 [00:01<00:03, 1109.06it/s]



 28%|██▊       | 1406/4996 [00:01<00:03, 1129.80it/s]



 30%|███       | 1523/4996 [00:01<00:03, 1138.11it/s]



 33%|███▎      | 1637/4996 [00:01<00:02, 1128.04it/s]



 35%|███▌      | 1754/4996 [00:01<00:02, 1138.52it/s]



 38%|███▊      | 1879/4996 [00:01<00:02, 1168.14it/s]



 40%|███▉      | 1996/4996 [00:01<00:02, 1152.22

In [ ]:
print("Accuracy =", 100 - np.sum(accuracy)/len(X_test))

Accuracy = 94.30745294872804
